In [2]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
module_path = os.path.abspath(os.path.join('..', 'src'))
sys.path.append(module_path)
import pandas as pd
import scipy.signal as ss
import sys

import tools.data_reader as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing

from scipy.fft import fft, fftfreq, fftshift
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [22]:
# SVM
importlib.reload(preprocessing)
importlib.reload(dr)
importlib.reload(dt)

import csv

metrics_folder = os.path.join(dr.Paths.DATA_DIR, "metrics")
metrics = ["bpm", "rmssd", "hf_rr", "lf_rr", "mean_SCL", "SCR_rate"]
phases = [
    dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH,
    dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT,
    dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT
]

clf = svm.SVC(kernel='poly', degree=8)

for metric in metrics:
    print(f"SVM USING {metric} -----------------------------------")
    ha = []
    la = []
    for phase in phases:
        file = os.path.join(metrics_folder, f"{metric}_{phase}_ha.csv")
        arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
        arr = arr[1:, 1:]
        # avg = np.nanmean(arr, axis=1)
        # ha.append(avg)
        arr = np.nan_to_num(arr)
        ha.append(arr)

        file = os.path.join(metrics_folder, f"{metric}_{phase}_la.csv")
        arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
        arr = arr[1:, 1:]
        # avg = np.nanmean(arr, axis=1)
        # la.append(avg)
        arr = np.nan_to_num(arr)
        la.append(arr)

    # ha_arr = np.asarray(ha).transpose()
    # la_arr = np.asarray(la).transpose()
    ha_arr = np.hstack(ha)
    la_arr = np.hstack(la)

    max_len = np.max([ha_arr.shape[1], la_arr.shape[1]])
    ha_arr = np.pad(ha_arr, ((0, 0), (0, max_len - ha_arr.shape[1])), "constant", constant_values=0.0)
    la_arr = np.pad(la_arr, ((0, 0), (0, max_len - la_arr.shape[1])), "constant", constant_values=0.0)

    x = np.vstack([ha_arr, la_arr])
    y = np.asarray([1 for _ in range(ha_arr.shape[0])] + [0 for _ in range(ha_arr.shape[0])])

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=16)

    clf.fit(x_train, y_train)

    preds = clf.predict(x_test)
    actual = y_test

    print(confusion_matrix(actual, preds))
    print(classification_report(actual, preds))

SVM USING bpm -----------------------------------
[[2 0]
 [2 2]]
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           1       1.00      0.50      0.67         4

    accuracy                           0.67         6
   macro avg       0.75      0.75      0.67         6
weighted avg       0.83      0.67      0.67         6

SVM USING rmssd -----------------------------------
[[2 0]
 [3 1]]
              precision    recall  f1-score   support

           0       0.40      1.00      0.57         2
           1       1.00      0.25      0.40         4

    accuracy                           0.50         6
   macro avg       0.70      0.62      0.49         6
weighted avg       0.80      0.50      0.46         6

SVM USING hf_rr -----------------------------------
[[2 0]
 [4 0]]
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         2
           1       0.00      0.00     

c:\Users\zhoux\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhoux\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhoux\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

[[0 2]
 [0 4]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.67      1.00      0.80         4

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6

SVM USING mean_SCL -----------------------------------


c:\Users\zhoux\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhoux\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhoux\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

[[2 0]
 [4 0]]
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         2
           1       0.00      0.00      0.00         4

    accuracy                           0.33         6
   macro avg       0.17      0.50      0.25         6
weighted avg       0.11      0.33      0.17         6

SVM USING SCR_rate -----------------------------------


c:\Users\zhoux\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhoux\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhoux\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

[[1 1]
 [0 4]]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.80      1.00      0.89         4

    accuracy                           0.83         6
   macro avg       0.90      0.75      0.78         6
weighted avg       0.87      0.83      0.81         6



In [25]:
# SVM
importlib.reload(preprocessing)
importlib.reload(dr)
importlib.reload(dt)

import csv

phases = [
    dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH,
    dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT,
    dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT
]

convert_sr = False
tasks = [dr.Tasks.BASELINE, dr.Tasks.BUGS, dr.Tasks.SPEAKING]
data_type = dr.DataTypes.ECG
fs = preprocessing.FS_DICT[data_type]
f_dim = preprocessing.DATA_TYPE_DIMENSIONS[data_type]

ha = []
la = []
for task in tasks:
    if task == dr.Tasks.BASELINE:
        phases = [dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH]
    elif task == dr.Tasks.BUGS:
        phases = [dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT]
    else:
        phases = [dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT]

    for phase in phases:
        ha_data, la_data = preprocessing.load_data(task, data_type, phase, convert_sr)
        ha_fft = [preprocessing.calculate_fft(data) for data in ha_data]
        la_fft = [preprocessing.calculate_fft(data) for data in la_data]
        dt.plot_fft_data(ha_fft[0])

[]


IndexError: list index out of range